# 旗揚げ ランダムパターン出力

In [1]:
import os
import random
import pandas as pd

COUNT_OF_SAMPLES = 100

HATA = ["赤","白"]
KAKEGOE_UP_DOWN = ["上げて","下げない","下げて","上げない"]
RESULT_PATTERN = ["赤","白","赤 白","どちらも上げない"]

In [2]:
def get_random_kakegoe1():
    kakegoe_all = ""
    for kake_count in range(8):
        hata = random.choice(HATA)
        up_down = random.choice(KAKEGOE_UP_DOWN)
        kakegoe1 = hata + up_down

        if kake_count%4 == 2 and (kakegoe1.find("ない")>=0):  # 3文字目が"ない"で終わる時は、"で"を付ける
            kakegoe1 += "で"
        
        if kake_count%4 < 3:
            kakegoe1 += "、"
        elif kake_count == 3:
            kakegoe1 += "\n"

        kakegoe_all += kakegoe1

    return kakegoe_all

test_kakegoe1 = get_random_kakegoe1()
test_kakegoe1

'赤下げて、白上げない、赤下げないで、赤下げて\n赤上げない、赤下げて、赤下げないで、赤下げて'

In [3]:
def solve(input_list):
    # 2行の文字列をsplitしながら受け取ります
    s = input_list[0].split('、') # 1行の文字列 → 4つの文字列のリストに変換
    s += input_list[1].split('、') # 1行の文字列 → 4つの文字列のリストに変換 → 前のリストに追加
    
    # 分解した８つの要素を、for文で回しながら解析していきます。
    red = False # 赤フラグ。Trueのときに赤を上げている状態として判定するための変数です
    white = False # 白フラグ。Trueのときに白を上げている状態として判定するための変数です
    for s1 in s: # 8つの要素を回すループ
        if s1[1:].startswith('上げて') or s1[1:].startswith('下げない'): # 2文字目以降を抽出して '上げて' もしくは '下げない' を判定します。つまり上げている状態の判定ですね。startswith()を用いているのは'で'が後ろにつくケースを考慮しています
            if s1[0] == '赤': # 1文字目が赤なら
                red = True # 赤フラグを立てます。赤を上げている状態となります。
            else: # 1文字目が赤以外なら白しかないので
                white = True # 白フラグを立てます。白を上げている状態となります。
        elif s1[1:].startswith('下げて') or s1[1:].startswith('上げない'): # 2文字目以降を抽出して '下げて' もしくは '上げない' を判定します。つまり下げている状態の判定ですね。startswith()を用いているのは'で'が後ろにつくケースを考慮しています
            if s1[0] == '赤': # 1文字目が赤なら
                red = False # 赤フラグをOFFにします。赤を下げている状態となります。
            else: # 1文字目が赤以外なら白しかないので
                white = False # 白フラグをOFFにします。白を下げている状態となります。
    #     print(red, white) # これは動作確認用のprint()文です。途中経過を見たい時にコメントアウトを外して、変数の内容を確認します。
    
    # 赤/白を上げた判定を元にprint文を書きます。「赤 白」「赤」「白」「どちらも上げない」の4パターンを出力します。
    if red and white:
        return ['赤 白']
    elif red:
        return ['赤']
    elif white:
        return ['白']
    else:
        return ['どちらも上げない']

test_output1 = solve(test_kakegoe1.split("\n"))
test_output1

['どちらも上げない']

In [8]:
def get_kakegoe2 (request_output, not_list=[]):
    for _ in range(257):
        test_kakegoe2 = get_random_kakegoe1()
        test_output2 = solve(test_kakegoe2)
        inputs = test_kakegoe2.split("\n")
        output_array = solve(inputs)
        if output_array[0] != request_output:
            # print("test_output({}) != request_output({})".format(output_array[0], request_output))
            continue
        if inputs in not_list:
            print("重複", inputs)
            continue

        print("result={}, kakegoe={}".format(request_output, test_kakegoe1))
        return inputs
    raise ValueError('無限ループしたようです')

get_kakegoe2("赤 白")

result=赤 白, kakegoe=赤下げて、白上げない、赤下げないで、赤下げて
赤上げない、赤下げて、赤下げないで、赤下げて


['赤上げて、赤下げない、白下げないで、白上げて', '赤下げない、白上げて、白上げて、赤下げない']

In [9]:
def get_random_inputs(not_list=[]):
    pattern_len = len(RESULT_PATTERN)
    answer_id = len(not_list)
    request_pattern_id = (answer_id % pattern_len)
    return get_kakegoe2(RESULT_PATTERN[request_pattern_id], not_list)
    
get_random_inputs()

result=赤, kakegoe=赤下げて、白上げない、赤下げないで、赤下げて
赤上げない、赤下げて、赤下げないで、赤下げて


['白上げない、赤下げて、白下げて、白下げない', '赤上げない、白上げない、赤上げて、白下げて']

In [10]:
def output_all(out_xz_filename):
    all_inputs = []
    all_outputs = []

    for answer_id in range(COUNT_OF_SAMPLES):
        all_inputs.append(get_random_inputs(all_inputs))
        all_outputs.append(solve(all_inputs[answer_id]))        # 解答を求める

    df_all = pd.DataFrame({"in":all_inputs, "out":all_outputs})
    df_all.index += 1
    os.makedirs(os.path.dirname(out_xz_filename), exist_ok=True)
    df_all.to_csv(out_xz_filename)

output_all("data/answer.csv.xz")
print('done')

result=赤, kakegoe=赤下げて、白上げない、赤下げないで、赤下げて
赤上げない、赤下げて、赤下げないで、赤下げて
result=白, kakegoe=赤下げて、白上げない、赤下げないで、赤下げて
赤上げない、赤下げて、赤下げないで、赤下げて
result=赤 白, kakegoe=赤下げて、白上げない、赤下げないで、赤下げて
赤上げない、赤下げて、赤下げないで、赤下げて
result=どちらも上げない, kakegoe=赤下げて、白上げない、赤下げないで、赤下げて
赤上げない、赤下げて、赤下げないで、赤下げて
result=赤, kakegoe=赤下げて、白上げない、赤下げないで、赤下げて
赤上げない、赤下げて、赤下げないで、赤下げて
result=白, kakegoe=赤下げて、白上げない、赤下げないで、赤下げて
赤上げない、赤下げて、赤下げないで、赤下げて
result=赤 白, kakegoe=赤下げて、白上げない、赤下げないで、赤下げて
赤上げない、赤下げて、赤下げないで、赤下げて
result=どちらも上げない, kakegoe=赤下げて、白上げない、赤下げないで、赤下げて
赤上げない、赤下げて、赤下げないで、赤下げて
result=赤, kakegoe=赤下げて、白上げない、赤下げないで、赤下げて
赤上げない、赤下げて、赤下げないで、赤下げて
result=白, kakegoe=赤下げて、白上げない、赤下げないで、赤下げて
赤上げない、赤下げて、赤下げないで、赤下げて
result=赤 白, kakegoe=赤下げて、白上げない、赤下げないで、赤下げて
赤上げない、赤下げて、赤下げないで、赤下げて
result=どちらも上げない, kakegoe=赤下げて、白上げない、赤下げないで、赤下げて
赤上げない、赤下げて、赤下げないで、赤下げて
result=赤, kakegoe=赤下げて、白上げない、赤下げないで、赤下げて
赤上げない、赤下げて、赤下げないで、赤下げて
result=白, kakegoe=赤下げて、白上げない、赤下げないで、赤下げて
赤上げない、赤下げて、赤下げないで、赤下げて
result=赤 白, kakegoe=赤下げて、白上げない、赤下げないで、赤下げて
赤上げない、赤下げて、赤下げないで、赤下げて
result=どちらも